<a href="https://colab.research.google.com/github/dhana-sekhar/Machine_Learning/blob/dev/LLM_Hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sekharpkg
!pip install langchain
!pip install openai
!pip install wikipedia
!pip install google-search-results

In [3]:
import openai
import langchain
from langchain.llms import OpenAI
from secret_key import openapi_key, serp_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key
os.environ['SERPAPI_API_KEY'] = serp_key

# Chains

In [9]:
llm = OpenAI(temperature=0.7)
name = llm("I want to open a restaurant for indian food. suggest me a fancy name for this")
print(name)



Spice Palace


In [8]:
from langchain.prompts import PromptTemplate

my_template = PromptTemplate(
    input_variables=['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. suggest me a fancy name for this"
)

my_prompt = my_template.format(cuisine = "mexican")

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=my_template)
chain.run("american")

'\n\n"Uncle Sam\'s Diner"'

In [11]:
my_template = PromptTemplate(
    input_variables=['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. suggest me a fancy name for this"
)

name_chain = LLMChain(llm=llm, prompt=my_template)

my_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template= "Suggest me some menu itesm for {restaurant_name}. return it a comma saperated values, dont exceed 5 items"
)

food_items_chain = LLMChain(llm=llm, prompt=my_template_items)

In [19]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Indian")
print(response)



Malai Kofta, Butter Chicken, Dal Makhani, Paneer Tikka Masala, Naan.


In [24]:
my_template = PromptTemplate(
    input_variables=['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. suggest me a fancy name for this"
)

name_chain = LLMChain(llm=llm, prompt=my_template, output_key = "restaurant_name")

my_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template= "Suggest me some menu itesm for {restaurant_name}. return it a comma saperated values, dont exceed 5 items"
)

food_items_chain = LLMChain(llm=llm, prompt=my_template_items, output_key="menu_items")

In [29]:
from langchain.chains import SequentialChain

schain = SequentialChain(
    chains = [ name_chain, food_items_chain],
    input_variables= ['cuisine'],
    output_variables=['restaurant_name', 'menu_items']
)

response = schain({'cuisine':'Indian'})

In [30]:
response

{'cuisine': 'Indian',
 'restaurant_name': '\n\nSpice Palace',
 'menu_items': '\n\nKorma, Tikka Masala, Biryani, Naan, Chana Masala'}

# Agents

In [4]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [5]:
llm = OpenAI(temperature=0.7)
tools = load_tools(['wikipedia', 'llm-math'], llm = llm)

my_agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

my_agent.run("when was elon musk born? what is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I need to find out when Elon Musk was born and how old he is now.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a business magnate and investor. Musk is the founder, chairman, CEO and chief technology officer of SpaceX;  angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of September 2023, according to the Bloomberg Billionaires Index, and $253 billion according to Forbes, primarily from his ownership stakes in both Tesla and SpaceX.Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother

'Elon Musk was born on June 28, 1971, and he is currently 52 years old in 2023.'

In [6]:
tools = load_tools(['serpapi', 'llm-math'], llm = llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("what was the GDP of US in 2022 plus 5")



> Entering new AgentExecutor chain...
 Is this a math question or something I need to search?
Action: Search
Action Input: GDP of US in 2022
Observation: $25.46 trillion
Thought: I need to add 5 to this number
Action: Calculator
Action Input: 25.46 + 5
Observation: Answer: 30.46
Thought: I now know the final answer
Final Answer: The GDP of US in 2022 plus 5 is $30.46 trillion.

> Finished chain.


'The GDP of US in 2022 plus 5 is $30.46 trillion.'

# Memory

In [12]:
from langchain.schema import prompt_template
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm = llm, prompt = my_template, memory = memory)
name = chain.run("mexican")
print(name)



Tacos y Salsas de la Familia


In [13]:
name = chain.run("Indian")
print(name)



Namaste Spice Kitchen


In [14]:
type(chain.memory)

langchain.memory.buffer.ConversationBufferMemory

In [15]:
chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='mexican'), AIMessage(content='\n\nTacos y Salsas de la Familia'), HumanMessage(content='Indian'), AIMessage(content='\n\nNamaste Spice Kitchen')]))

In [17]:
print(chain.memory.buffer)

Human: mexican
AI: 

Tacos y Salsas de la Familia
Human: Indian
AI: 

Namaste Spice Kitchen


In [32]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm = llm)
convo.prompt

PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [33]:
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [34]:
convo.run("who won the first cricket world cup")

' The first Cricket World Cup was held in 1975 and was won by the West Indies.'

In [35]:
convo.run("what is 5+5")

' The sum of 5 and 5 is 10.'

In [36]:
convo.run("who is the captain of the winning team")

' The captain of the West Indies team that won the first Cricket World Cup was Clive Lloyd.'

In [37]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
convo = ConversationChain(
    llm = llm,
    memory = memory
)
convo.run("who won the first cricket world cup")

' The first Cricket World Cup was won by the West Indies, under the captaincy of Clive Lloyd, in 1975.'

In [38]:
convo.run("what is 5+5")

' The sum of 5 + 5 is 10.'

In [39]:
convo.run("who is the captain of the winning team")

" I'm sorry, I don't know."

In [40]:
convo.memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='who won the first cricket world cup'), AIMessage(content=' The first Cricket World Cup was won by the West Indies, under the captaincy of Clive Lloyd, in 1975.'), HumanMessage(content='what is 5+5'), AIMessage(content=' The sum of 5 + 5 is 10.'), HumanMessage(content='who is the captain of the winning team'), AIMessage(content=" I'm sorry, I don't know.")]), k=1)

In [41]:
print(convo.memory.buffer)

Human: who is the captain of the winning team
AI:  I'm sorry, I don't know.
